In [1]:
import itertools
import random

import IPython.display
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# df_orig = pd.read_csv('../logs/rundirs/sorted-20241116_215630.csv')
df_orig = pd.read_csv('../logs/rundirs/sorted-20241119_223637.csv')
df_orig

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s)
0,20241119_213822,map-generator/generated-maps/2024-11-16_17:00:...,0,HumanDrivenVehicle,77.56,14,1152.9,24,0,173,0,1,0,57.4,19.3,599.9,0.3,5.6,1.9
1,20241119_214026,map-generator/generated-maps/2024-11-16_17:00:...,0,HumanDrivenVehicle,77.56,19,1463.1,19,20,173,3,6,0,14.4,1.1,599.9,0.3,5.6,2.4
2,20241119_214227,map-generator/generated-maps/2024-11-16_17:00:...,0,HumanDrivenVehicle,77.56,19,1457.5,19,18,183,7,9,0,14.4,1.1,599.9,0.3,5.6,2.4
3,20241119_214435,map-generator/generated-maps/2024-11-16_17:00:...,0,HumanDrivenVehicle,77.56,10,805.8,25,0,176,0,1,0,151.7,22.1,599.9,0.3,5.6,1.3
4,20241119_214625,map-generator/generated-maps/2024-11-16_17:00:...,0,HumanDrivenVehicle,77.56,17,1324.9,19,26,215,2,5,0,25.0,11.1,599.9,0.3,5.6,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,20241119_222630,map-generator/generated-maps/2024-11-16_17:00:...,4,AutonomousVehicle,60.34,15,1914.8,15,0,90,0,2,0,11.2,0.7,599.9,0.3,5.6,3.2
146,20241119_222829,map-generator/generated-maps/2024-11-16_17:00:...,4,AutonomousVehicle,60.34,10,1252.3,21,0,73,3,1,0,49.0,16.0,599.9,0.3,5.6,2.1
147,20241119_223021,map-generator/generated-maps/2024-11-16_17:00:...,4,AutonomousVehicle,52.64,12,1282.5,14,0,141,1,6,0,29.0,19.7,599.9,0.3,5.6,2.1
148,20241119_223226,map-generator/generated-maps/2024-11-16_17:00:...,4,AutonomousVehicle,52.64,12,1281.7,15,0,143,1,8,0,29.3,19.7,599.9,0.3,5.6,2.1


In [3]:
df_id = df_orig['Scenario ID'].str.split(r'[;,] ', expand=True)
df_id.columns = ['filename', 'Coordination strategy', 'string_seed', 'string_probabilityForcingForHuman', 'heuristic']
df_id = pd.concat([
    df_id, 
    df_id['filename'].str.extract(r'/scenario(?P<i_map>\d+)-(?P<i_locations>\d+).json$', expand=True).astype(int),
    df_id['string_probabilityForcingForHuman'].str.extract(r'^probabilityForcingForHuman (?P<probabilityForcingForHuman>[\d.]+)$', expand=True).astype(float),
], axis=1).rename(columns={'i_locations': 'Positions variant'})
df_id

,filename,Coordination strategy,string_seed,string_probabilityForcingForHuman,heuristic,i_map,Positions variant,probabilityForcingForHuman
0,map-generator/generated-maps/2024-11-16_17:00:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,1,1,0.0
1,map-generator/generated-maps/2024-11-16_17:00:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,1,1,1.0
2,map-generator/generated-maps/2024-11-16_17:00:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,1,1,1.0
3,map-generator/generated-maps/2024-11-16_17:00:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,1,2,0.0
4,map-generator/generated-maps/2024-11-16_17:00:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,1,2,1.0
...,...,...,...,...,...,...,...,...
145,map-generator/generated-maps/2024-11-16_17:00:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,1,9,1.0
146,map-generator/generated-maps/2024-11-16_17:00:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,1,9,1.0
147,map-generator/generated-maps/2024-11-16_17:00:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,1,10,0.0
148,map-generator/generated-maps/2024-11-16_17:00:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,1,10,1.0


In [4]:
index_id = df_orig.columns.get_loc('Scenario ID')
df = pd.concat([df_id[['Coordination strategy', 'probabilityForcingForHuman', 'Positions variant']], df_orig.iloc[:, index_id + 1:]], axis=1)
df

,Coordination strategy,probabilityForcingForHuman,Positions variant,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s)
0,baseline,0.0,1,0,HumanDrivenVehicle,77.56,14,1152.9,24,0,173,0,1,0,57.4,19.3,599.9,0.3,5.6,1.9
1,change of priorities,1.0,1,0,HumanDrivenVehicle,77.56,19,1463.1,19,20,173,3,6,0,14.4,1.1,599.9,0.3,5.6,2.4
2,stops,1.0,1,0,HumanDrivenVehicle,77.56,19,1457.5,19,18,183,7,9,0,14.4,1.1,599.9,0.3,5.6,2.4
3,baseline,0.0,2,0,HumanDrivenVehicle,77.56,10,805.8,25,0,176,0,1,0,151.7,22.1,599.9,0.3,5.6,1.3
4,change of priorities,1.0,2,0,HumanDrivenVehicle,77.56,17,1324.9,19,26,215,2,5,0,25.0,11.1,599.9,0.3,5.6,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,change of priorities,1.0,9,4,AutonomousVehicle,60.34,15,1914.8,15,0,90,0,2,0,11.2,0.7,599.9,0.3,5.6,3.2
146,stops,1.0,9,4,AutonomousVehicle,60.34,10,1252.3,21,0,73,3,1,0,49.0,16.0,599.9,0.3,5.6,2.1
147,baseline,0.0,10,4,AutonomousVehicle,52.64,12,1282.5,14,0,141,1,6,0,29.0,19.7,599.9,0.3,5.6,2.1
148,change of priorities,1.0,10,4,AutonomousVehicle,52.64,12,1281.7,15,0,143,1,8,0,29.3,19.7,599.9,0.3,5.6,2.1


In [5]:
series_deadlocks = df[df['Positions variant'] != 11].groupby(['Positions variant'])['Is blocked'].sum()
indices_locations_deadlocked = series_deadlocks[series_deadlocks != 0].index
indices_locations_nondeadlocked = series_deadlocks[series_deadlocks == 0].index
indices_locations_nondeadlocked

Index([1, 2, 3, 5, 6, 7, 8, 9, 10], dtype='int64', name='Positions variant')

In [6]:
def save_and_show(filename):  # to avoid inlining large image data into the notebook file
    plt.savefig(filename)
    plt.close()
    # The `random` is because of https://stackoverflow.com/a/43640705.
    IPython.display.display(IPython.display.HTML(f'<img src="{filename}?{random.random()}" alt="{filename}" />'))

In [7]:
def plot_aut(df):
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    # Plot configuration
    fig, axes = plt.subplots(1, len(positions), figsize=(20, 6), sharey=True, squeeze=False)
    axes: list[list[Axes]]
    bar_width = 0.4
    colors = ['tab:blue', 'red', 'yellow']
    
    # Get the global min and max values for consistent y-axis scaling
    y1_min, y1_max = df['Total distance traveled (m)'].min(), df['Total distance traveled (m)'].max()
    y2_min, y2_max = min(df['No. of collisions'].min(), df['No. of near-misses'].min()), max(df['No. of collisions'].max(), df['No. of near-misses'].max())
    
    # Iterate through each Positions variant
    for i, pos in enumerate(positions):
        ax = axes[0][i]
        df_pos = df.loc[pos]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars for 'Total distance traveled (m)'
        bars = ax.bar(x_positions, df_pos['Total distance traveled (m)'], width=bar_width, label='Distance Traveled', color=colors[0])
        ax.set_xlabel('Coordination Strategy')
        ax.set_ylabel('Total Distance Traveled (m)', color=colors[0])
        ax.tick_params(axis='y', labelcolor=colors[0])
        ax.set_ylim(0, y1_max * 1.1)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points for 'No. of collisions' and 'No. of near-misses'
        collisions, = ax_right.plot(x_positions, df_pos['No. of collisions'], marker='o', linestyle='', label='No. of Collisions', color=colors[1])
        near_misses, = ax_right.plot(x_positions, df_pos['No. of near-misses'], marker='o', linestyle='', label='No. of Near Misses', color=colors[2])
        ax_right.set_ylabel('Collisions (r) & Near Misses (y)', color='black')
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_max * 1.1)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels(strategies, rotation=45, ha='right')
        ax.set_title(f'Positions variant {pos}')
        ax.grid(axis='y')
        
        if i == 0:
            handles = [bars, collisions, near_misses]
            labels = ['Distance Traveled', 'No. of Collisions', 'No. of Near Misses']
        
    fig.suptitle('Automated vehicles (summarised for AV1-AV4)', fontsize=16)
    fig.legend(handles=handles, labels=labels, loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.1))
    plt.tight_layout()    
    save_and_show('aut20kmh.png')

    
df_aut = df
df_aut = df_aut[df_aut['Positions variant'].isin(indices_locations_nondeadlocked)]
df_aut = df_aut[df_aut['Vehicle type'] == 'AutonomousVehicle']
df_aut = df_aut.groupby(['Positions variant', 'Coordination strategy']).sum()
df_aut = df_aut[['Total distance traveled (m)', 'No. of collisions', 'No. of near-misses']]
plot_aut(df_aut)
df_aut

Total distance traveled (m)  \
Positions variant Coordination strategy                                
1                 baseline                                    6224.5   
                  change of priorities                        6198.2   
                  stops                                       5583.0   
2                 baseline                                    5725.5   
                  change of priorities                        5671.9   
                  stops                                       4966.8   
3                 baseline                                    5239.3   
                  change of priorities                        4920.7   
                  stops                                       4636.2   
5                 baseline                                    5568.6   
                  change of priorities                        5272.2   
                  stops                                       5096.4   
6                 baseline                                    6512.0   
                  change of priorities                        6419.4   
                  stops                                       5267.6   
7                 baseline                                    6056.6   
                  change of priorities                        5873.2   
                  stops                                       4617.2   
8                 baseline                                    6079.7   
                  change of priorities                        5972.0   
                  stops                                       5357.3   
9                 baseline                                    5770.7   
                  change of priorities                        6123.9   
                  stops                                       4758.5   
10                baseline                                    5965.7   
                  change of priorities                        5963.7   
                  stops                                       5632.3   

                                         No. of collisions  No. of near-misses  
Positions variant Coordination strategy                                         
1                 baseline                               1                   0  
                  change of priorities                   6                   3  
                  stops                                  9                   7  
2                 baseline                               1                   0  
                  change of priorities                   5                   2  
                  stops                                 11                   5  
3                 baseline                               4                   5  
                  change of priorities                   3                   2  
                  stops                                  2                   1  
5                 baseline                               2                   0  
                  change of priorities                   2                   0  
                  stops                                 11                   4  
6                 baseline                               0                   0  
                  change of priorities                   5                   2  
                  stops                                  4                   1  
7                 baseline                               0                   0  
                  change of priorities                   1                   2  
                  stops                                  7                   0  
8                 baseline                               9                   5  
                  change of priorities                   4                   7  
                  stops                                  3                   3  
9                 baseline                               0                   4  
        

In [8]:
def plot_hum(df):
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    # Plot configuration
    fig, axes = plt.subplots(1, len(positions), figsize=(20, 6), sharey=True, squeeze=False)
    axes: list[list[Axes]]
    bar_width = 0.4
    colors = ['tab:blue', 'black', 'red', 'yellow']
    
    # Get the global min and max values for consistent y-axis scaling
    y1_min, y1_max = df['Total distance traveled (m)'].min(), df['Total distance traveled (m)'].max()
    # y2_min, y2_max = df['No. of forcing events'].min(), df['No. of forcing events'].max()
    y2_min, y2_max = min(df['No. of forcing events'].min(), df['No. of collisions'].min(), df['No. of near-misses'].min()), max(df['No. of forcing events'].max(), df['No. of collisions'].max(), df['No. of near-misses'].max())
    
    # Iterate through each Positions variant
    for i, pos in enumerate(positions):
        ax = axes[0][i]
        df_pos = df.loc[pos]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars for 'Total distance traveled (m)'
        bars = ax.bar(x_positions, df_pos['Total distance traveled (m)'], width=bar_width, label='Distance Traveled', color=colors[0])
        ax.set_xlabel('Coordination Strategy')
        ax.set_ylabel('Total Distance Traveled (m)', color=colors[0])
        ax.tick_params(axis='y', labelcolor=colors[0])
        ax.set_ylim(0, y1_max * 1.1)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points for 'No. of collisions' and 'No. of near-misses'
        forcing_events, = ax_right.plot(x_positions, df_pos['No. of forcing events'], marker='o', linestyle='', label='No. of forcing events', color=colors[1])
        collisions, = ax_right.plot(x_positions, df_pos['No. of collisions'], marker='o', linestyle='', label='No. of Collisions', color=colors[2])
        near_misses, = ax_right.plot(x_positions, df_pos['No. of near-misses'], marker='o', linestyle='', label='No. of Near Misses', color=colors[3])
        ax_right.set_ylabel('forcing events (b), collisions (r) & near misses (y)', color='black')
        # ax_right.set_ylabel('Collisions & Near Misses', color='black')
    
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_max * 1.1)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels(strategies, rotation=45, ha='right')
        ax.set_title(f'Positions variant {pos}')
        ax.grid(axis='y')
        
        if i == 0:
            handles = [bars, forcing_events, collisions, near_misses]
            labels = ['Distance Traveled', 'No. of forcing events', 'No. of Collisions', 'No. of Near Misses']
        
    fig.suptitle('Human-driven vehicle', fontsize=16)
    fig.legend(handles=handles, labels=labels, loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.1))
    plt.tight_layout()
    save_and_show('hum20kmh1.png')

    
df_hum = df
df_hum = df_hum[df_hum['Positions variant'].isin(indices_locations_nondeadlocked)]
df_hum = df_hum[df_hum['Vehicle type'] == 'HumanDrivenVehicle']
df_hum = df_hum.groupby(['Positions variant', 'Coordination strategy']).sum()
df_hum = df_hum[['Total distance traveled (m)', 'No. of forcing events', 'No. of collisions', 'No. of near-misses']]
plot_hum(df_hum)
df_hum

Total distance traveled (m)  \
Positions variant Coordination strategy                                
1                 baseline                                    1152.9   
                  change of priorities                        1463.1   
                  stops                                       1457.5   
2                 baseline                                     805.8   
                  change of priorities                        1324.9   
                  stops                                       1398.3   
3                 baseline                                    1100.6   
                  change of priorities                        1459.6   
                  stops                                       1465.3   
5                 baseline                                    1092.4   
                  change of priorities                        1383.3   
                  stops                                       1436.6   
6                 baseline                                     709.6   
                  change of priorities                        1316.7   
                  stops                                       1385.9   
7                 baseline                                     289.7   
                  change of priorities                        1380.5   
                  stops                                       1465.2   
8                 baseline                                     791.8   
                  change of priorities                        1379.5   
                  stops                                       1458.5   
9                 baseline                                     738.8   
                  change of priorities                        1255.0   
                  stops                                       1384.4   
10                baseline                                    1437.8   
                  change of priorities                        1438.0   
                  stops                                       1461.7   

                                         No. of forcing events  \
Positions variant Coordination strategy                          
1                 baseline                                   0   
                  change of priorities                      20   
                  stops                                     18   
2                 baseline                                   0   
                  change of priorities                      26   
                  stops                                     28   
3                 baseline                                   0   
                  change of priorities                      11   
                  stops                                     14   
5                 baseline                                   0   
                  change of priorities                      18   
                  stops                                     20   
6                 baseline                                   0   
                  change of priorities                      22   
                  stops                                     32   
7                 baseline                                   0   
                  change of priorities                      32   
                  stops                                     29   
8                 baseline                                   0   
                  change of priorities                      14   
                  stops                                     17   
9                 baseline                                   0   
                  change of priorities                      27   
                  stops                                     25   
10                baseline                                   0   
                  change of priorities                       8   
                  stops                                      8   

                                     

In [9]:
def show_maps(title, indices, ncols): 
    nrows = (len(indices) + ncols - 1) // ncols
    fig, axes_matrix = plt.subplots(nrows, ncols, figsize=(16, 9), squeeze=False)
    #print(f'{title}: {nrows}x{ncols}')

    axes = list(itertools.chain.from_iterable(axes_matrix))
    axes_matrix: list[list[Axes]]
    assert len(axes) >= len(indices)
    
    for ax in axes:
        ax.axis('off')
        
    for ax, i_pos in zip(axes, indices):
        image = plt.imread(f'../screenshotting/screenshots/cropped/{i_pos}.png')
        ax.imshow(image)
        ax.title.set_text(f'Positions variant {i_pos}')
    
    fig.suptitle(title, fontsize=16)

    plt.subplots_adjust(wspace=0.1, hspace=0.3)
    plt.tight_layout()
    save_and_show(title + '.png')
    
show_maps('Non-deadlocked20kmh', indices_locations_nondeadlocked, 4)
show_maps('Deadlocked20kmh', indices_locations_deadlocked, 4)